In [1]:
import pathlib

import pandas as pd

In [2]:
file_path = pathlib.Path(
    "../../data/5.converted_data/sum_aggregated_data.parquet"
).resolve(strict=True)
df = pd.read_parquet(file_path)
df.head()

,Metadata_genotype,Metadata_replicate,AreaShape_Area,AreaShape_CentralMoment_0_0,AreaShape_CentralMoment_0_1,AreaShape_CentralMoment_0_2,AreaShape_CentralMoment_0_3,AreaShape_CentralMoment_1_0,AreaShape_CentralMoment_1_1,AreaShape_CentralMoment_1_2,...,Texture_SumEntropy_OP_3_02_256,Texture_SumEntropy_OP_3_03_256,Texture_SumVariance_OP_3_00_256,Texture_SumVariance_OP_3_01_256,Texture_SumVariance_OP_3_02_256,Texture_SumVariance_OP_3_03_256,Texture_Variance_OP_3_00_256,Texture_Variance_OP_3_01_256,Texture_Variance_OP_3_02_256,Texture_Variance_OP_3_03_256
0,high,1,1.192966,1.192966,0.541753,-0.098475,0.781736,4.201830,-0.927874,-0.085964,...,-0.208210,-0.238073,-1.091712,-1.050054,-1.086609,-1.089178,-1.115127,-1.124203,-1.117953,-1.121744
1,high,10,2.477908,2.477908,1.319626,3.444728,-1.748595,0.358390,-3.289519,1.650540,...,1.293954,1.270712,1.261364,1.284182,1.187508,1.270548,1.115726,1.096947,1.121969,1.095539
2,high,11,4.262414,4.262414,1.261731,2.852939,0.583182,3.602414,-0.541325,-0.328613,...,1.373542,1.356461,1.104765,1.141646,1.052532,1.096949,0.997651,0.972537,1.000166,0.980426
3,high,12,2.402476,2.402476,0.049886,2.558238,-0.928171,-3.305706,-1.964619,0.215359,...,0.619009,0.617867,-0.109557,-0.119752,-0.196800,-0.135860,-0.207564,-0.218735,-0.202171,-0.213544
4,high,13,2.389777,2.389777,-1.108736,-0.068931,0.101765,-2.689960,-0.967345,-1.588868,...,0.113413,0.095483,-0.650341,-0.591348,-0.642077,-0.644871,-0.696817,-0.712509,-0.700572,-0.704335


We want to remove CellProfiler features that do not accurately contribute to the shape variance.


Spatial moment features are removed as they are not invariant to rotation and translation.

Central moment features are removed as measurements of size is not of interest.

Normalized moment features are removed as they do not contribute to the shape variance.

EulerNumber is removed as we are not concerned with the number of holes in the shape.

Extent is removed as we do not care about the ratio of pixels in the bounding box to the pixels in the shape.

Orientation is removed as we are not concerned with the angle of the major axis of the shape.

Zernike features are removed as they are not invariant to rotation and translation.

Texture features are removed as we are not interested in the texture of the shape.

In [3]:
list_to_drop = [
    "SpatialMoment",
    "CentralMoment",
    "NormalizedMoment",
    "EulerNumber",
    "Extent",
    "Orientation",
    "Zernike",
    "Texture",
]

In [4]:
# drop columns that contain Metadata
df1 = df.copy()
df1 = df1.drop(columns=[col for col in df1.columns if "Metadata" in col])
columns = df1.columns.to_list()
print(df1.shape)
# if column contains any of the strings in list_to_drop, drop it
for col in list_to_drop:
    columns = [c for c in columns if col not in c]
df1 = df1[columns]
print(df1.shape)
df1["Metadata_genotype"] = df["Metadata_genotype"]
df1["Metadata_replicate"] = df["Metadata_replicate"]

(42, 244)
(42, 64)


In [5]:
df1.head()

,AreaShape_Area,AreaShape_Compactness,AreaShape_ConvexArea,AreaShape_Eccentricity,AreaShape_EquivalentDiameter,AreaShape_FormFactor,AreaShape_HuMoment_0,AreaShape_HuMoment_1,AreaShape_HuMoment_2,AreaShape_HuMoment_3,...,RadialDistribution_MeanFrac_OP_1of4,RadialDistribution_MeanFrac_OP_2of4,RadialDistribution_MeanFrac_OP_3of4,RadialDistribution_MeanFrac_OP_4of4,RadialDistribution_RadialCV_OP_1of4,RadialDistribution_RadialCV_OP_2of4,RadialDistribution_RadialCV_OP_3of4,RadialDistribution_RadialCV_OP_4of4,Metadata_genotype,Metadata_replicate
0,1.192966,0.923269,1.744956,-4.171096,1.348456,-0.007464,-1.438887,-1.339881,-1.604103,-1.285406,...,0.208840,-0.077966,-0.205642,0.361141,-1.659848,-1.496419,-0.734737,-0.894294,high,1
1,2.477908,-2.169614,2.280730,-0.195609,2.496823,2.557252,-1.530468,-1.217577,-1.845856,-1.282933,...,-2.497710,-2.751145,-2.599793,3.112188,1.782285,1.249337,2.211775,2.804916,high,10
2,4.262414,-1.510935,4.433965,-3.772736,3.782522,1.591618,-1.848755,-1.370704,-1.922923,-1.321971,...,0.143854,-1.042653,-1.791332,1.456908,1.205187,1.159028,1.059906,1.368864,high,11
3,2.402476,-2.343360,2.190627,-0.593213,2.427407,2.977069,-1.480895,-1.185775,-1.731990,-1.215138,...,-2.176503,-2.487337,-1.981686,2.545706,-0.908438,-0.178470,1.397076,1.672961,high,12
4,2.389777,-1.904166,2.200426,0.105454,2.394630,2.050486,-1.305085,-1.097574,-1.570746,-1.050117,...,-2.530155,-2.831366,-2.696436,3.358448,-1.159343,-1.876114,0.004541,1.253884,high,13


In [6]:
# save the dataframe
df_path = pathlib.Path(
    "../../data/5.converted_data/sum_aggregated_manual_feature_selected_data.parquet"
)
df1.to_parquet(df_path)